This exercise is adapted from https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/


## Keras Tutorial Overview

There is not a lot of code required, but we are going to step over it slowly so that you will know how to create your own models in the future.

The steps you are going to cover in this tutorial are as follows:

1. Load Data.
2. Define Keras Model.
3. Compile Keras Model.
4. Fit Keras Model.
5. Evaluate Keras Model.
6. Tie It All Together.
7. Make Predictions

This Keras tutorial has a few requirements:

1. You have Python 2 or 3 installed and configured.
2. You have SciPy (including NumPy) installed and configured.
3. You have Keras and a backend (Theano or TensorFlow) installed and configured.

### 1. Load Data
The first step is to define the functions and classes we intend to use in this tutorial.

We will use the NumPy library to load our dataset and we will use two classes from the Keras library to define our model.

The imports required are listed below.

In [1]:
!pip install seaborn

In [2]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense


import seaborn as sns

sns.set()


Now load the data from *file pima-indians-diabetes.data.csv* stored in Data folder (check Google Drive if unsure)

In [3]:
# load the dataset
dataset = loadtxt('.\Data\pima-indians-diabetes.csv', delimiter=',')

# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

**Q:** Find out more about the dataset - such as how many columns, what kind of information does it provide etc.

In [4]:
import pandas as pd

df = pd.DataFrame(data=dataset)
# df.head()
df.describe()


,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    float64
 1   1       768 non-null    float64
 2   2       768 non-null    float64
 3   3       768 non-null    float64
 4   4       768 non-null    float64
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    float64
 8   8       768 non-null    float64
dtypes: float64(9)
memory usage: 54.1 KB


### 2. Define Keras Model
Models in Keras are defined as a sequence of layers.

We create a **Sequential model** and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the **input_dim** argument and setting it to 8 for the 8 input variables.

*Question:* How do we know the number of layers and their types?

This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation (I explain more about this here). Generally, you need a network large enough to capture the structure of the problem.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the activation argument.

We will use the rectified linear unit activation function referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer:

* The model expects rows of data with 8 variables (the input_dim=8 argument)
* The first hidden layer has 12 nodes and uses the relu activation function.
* The second hidden layer has 8 nodes and uses the relu activation function.
* The output layer has one node and uses the sigmoid activation function.

In [6]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### 3. Compile Keras Model
Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in our dataset.

We must specify the loss function to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use cross entropy as the **loss** argument. This loss is for a binary classification problems and is defined in Keras as **“binary_crossentropy“**. 

We will define the optimizer as the efficient stochastic gradient descent algorithm “adam“. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems. 

Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the metrics argument.

In [7]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. Fit Keras Model
We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

Training occurs over epochs and each epoch is split into batches.

* **Epoch:** One pass through all of the rows in the training dataset.
* **Batch:** One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the epochs argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the batch_size argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.

In [8]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 0s 1ms/step - loss: 7.1550 - accuracy: 0.4036
Epoch 2/150
77/77 [==============================] - 0s 969us/step - loss: 1.7357 - accuracy: 0.5417
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7236 - accuracy: 0.6432
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7053 - accuracy: 0.6667
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6859 - accuracy: 0.6693
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6697 - accuracy: 0.6823
Epoch 7/150
77/77 [==============================] - 0s 961us/step - loss: 0.6647 - accuracy: 0.6914
Epoch 8/150
77/77 [==============================] - 0s 974us/step - loss: 0.6481 - accuracy: 0.6940
Epoch 9/150
77/77 [==============================] - 0s 948us/step - loss: 0.6399 - accuracy: 0.7018
Epoch 10/150
77/77 [==============================] - 0s 961us/step - loss: 0.6381 - accuracy: 0.7083

77/77 [==============================] - 0s 907us/step - loss: 0.5055 - accuracy: 0.7656
Epoch 83/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5066 - accuracy: 0.7708
Epoch 84/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4964 - accuracy: 0.7708
Epoch 85/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5008 - accuracy: 0.7617
Epoch 86/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4943 - accuracy: 0.7826
Epoch 87/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4974 - accuracy: 0.7630
Epoch 88/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4933 - accuracy: 0.7656
Epoch 89/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4968 - accuracy: 0.7591
Epoch 90/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4956 - accuracy: 0.7617
Epoch 91/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5042 - accuracy: 0.7656
Epoch 92/15

**Q:** change the value of Epoch - what differences will you expect/see?

In [9]:
# fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=30)

Epoch 1/300
26/26 [==============================] - 0s 5ms/step - loss: 0.4593 - accuracy: 0.7865
Epoch 2/300
26/26 [==============================] - 0s 4ms/step - loss: 0.4563 - accuracy: 0.7930
Epoch 3/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4569 - accuracy: 0.7878
Epoch 4/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4557 - accuracy: 0.7917
Epoch 5/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4545 - accuracy: 0.7943
Epoch 6/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4551 - accuracy: 0.7969
Epoch 7/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4548 - accuracy: 0.7917
Epoch 8/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4546 - accuracy: 0.7969
Epoch 9/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4554 - accuracy: 0.7852
Epoch 10/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4566 - accuracy: 0.7852
Epoch 11/

26/26 [==============================] - 0s 3ms/step - loss: 0.4459 - accuracy: 0.7969
Epoch 83/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4499 - accuracy: 0.7878
Epoch 84/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4524 - accuracy: 0.7826
Epoch 85/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4400 - accuracy: 0.7917
Epoch 86/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4486 - accuracy: 0.8021
Epoch 87/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4450 - accuracy: 0.7878
Epoch 88/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4458 - accuracy: 0.7878
Epoch 89/300
26/26 [==============================] - 0s 921us/step - loss: 0.4409 - accuracy: 0.7943
Epoch 90/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4398 - accuracy: 0.8021
Epoch 91/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4394 - accuracy: 0.7956
Epoch 92/30

26/26 [==============================] - 0s 2ms/step - loss: 0.4310 - accuracy: 0.8034
Epoch 164/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4347 - accuracy: 0.7930
Epoch 165/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4338 - accuracy: 0.8047
Epoch 166/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4304 - accuracy: 0.8047
Epoch 167/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4303 - accuracy: 0.8034
Epoch 168/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4294 - accuracy: 0.8099
Epoch 169/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4311 - accuracy: 0.8034
Epoch 170/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4277 - accuracy: 0.8099
Epoch 171/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.8086
Epoch 172/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4309 - accuracy: 0.8021
Epoc

26/26 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.8086
Epoch 245/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4264 - accuracy: 0.8073
Epoch 246/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4328 - accuracy: 0.8034
Epoch 247/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4391 - accuracy: 0.8008
Epoch 248/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4323 - accuracy: 0.7930
Epoch 249/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4215 - accuracy: 0.8060
Epoch 250/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4225 - accuracy: 0.8203
Epoch 251/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4188 - accuracy: 0.8073
Epoch 252/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4250 - accuracy: 0.8086
Epoch 253/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4196 - accuracy: 0.8060
Epoc

### 5. Evaluate Keras Model
We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the **evaluate()** function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The **evaluate()** function will return a list with two values. The first will be the loss of the model on the dataset and the second will be the accuracy of the model on the dataset. We are only interested in reporting the accuracy, so we will ignore the loss value.

In [10]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 955us/step - loss: 0.4217 - accuracy: 0.8151
Accuracy: 81.51


### 7. Make Predictions
We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before.

Making predictions is as easy as calling the predict() function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [11]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions 
rounded = [round(x[0]) for x in predictions]


**Q:** now redo predictions by using the function predict_classes(). Show the outcome of the first 10 cases (code provided)

In [12]:
import numpy as np


classes_x=np.argmax(predictions,axis=1)


for i in range(10):
    print('%s => %d (expected %d)' % (X[i].tolist(), classes_x[i], y[i]))


[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] => 0 (expected 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] => 0 (expected 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] => 0 (expected 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] => 0 (expected 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] => 0 (expected 1)


## Challenge 1:
Separate the training and test datasets (70% vs 30%) and rerun the evaluation

In [13]:
!pip install sklearn

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [16]:
# fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=30)

Epoch 1/300
26/26 [==============================] - 0s 1ms/step - loss: 17.3595 - accuracy: 0.6510
Epoch 2/300
26/26 [==============================] - 0s 1ms/step - loss: 7.2400 - accuracy: 0.6107
Epoch 3/300
26/26 [==============================] - 0s 1ms/step - loss: 3.5860 - accuracy: 0.5156
Epoch 4/300
26/26 [==============================] - 0s 1ms/step - loss: 2.1486 - accuracy: 0.5365
Epoch 5/300
26/26 [==============================] - 0s 2ms/step - loss: 1.4002 - accuracy: 0.5195
Epoch 6/300
26/26 [==============================] - 0s 1ms/step - loss: 1.1219 - accuracy: 0.5651
Epoch 7/300
26/26 [==============================] - 0s 1ms/step - loss: 0.9812 - accuracy: 0.5885
Epoch 8/300
26/26 [==============================] - 0s 2ms/step - loss: 0.8967 - accuracy: 0.5938
Epoch 9/300
26/26 [==============================] - 0s 1ms/step - loss: 0.8391 - accuracy: 0.5951
Epoch 10/300
26/26 [==============================] - 0s 1ms/step - loss: 0.8078 - accuracy: 0.6081
Epoch 11

26/26 [==============================] - 0s 962us/step - loss: 0.5512 - accuracy: 0.7344
Epoch 83/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5947 - accuracy: 0.7005
Epoch 84/300
26/26 [==============================] - 0s 961us/step - loss: 0.5502 - accuracy: 0.7370
Epoch 85/300
26/26 [==============================] - 0s 960us/step - loss: 0.5669 - accuracy: 0.7083
Epoch 86/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5429 - accuracy: 0.7370
Epoch 87/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5537 - accuracy: 0.7201
Epoch 88/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5578 - accuracy: 0.7214
Epoch 89/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5687 - accuracy: 0.7174
Epoch 90/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5476 - accuracy: 0.7318
Epoch 91/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5508 - accuracy: 0.7266
Epoch 9

26/26 [==============================] - 0s 1ms/step - loss: 0.5065 - accuracy: 0.7617
Epoch 164/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5143 - accuracy: 0.7474
Epoch 165/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5207 - accuracy: 0.7604
Epoch 166/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4929 - accuracy: 0.7669
Epoch 167/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5173 - accuracy: 0.7422
Epoch 168/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5060 - accuracy: 0.7461
Epoch 169/300
26/26 [==============================] - 0s 920us/step - loss: 0.4960 - accuracy: 0.7617
Epoch 170/300
26/26 [==============================] - 0s 1ms/step - loss: 0.5065 - accuracy: 0.7500
Epoch 171/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4977 - accuracy: 0.7526
Epoch 172/300
26/26 [==============================] - 0s 960us/step - loss: 0.4978 - accuracy: 0.7474


26/26 [==============================] - 0s 1ms/step - loss: 0.4900 - accuracy: 0.7708
Epoch 244/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4780 - accuracy: 0.7799
Epoch 245/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4977 - accuracy: 0.7643
Epoch 246/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4664 - accuracy: 0.7799
Epoch 247/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4710 - accuracy: 0.7708
Epoch 248/300
26/26 [==============================] - 0s 964us/step - loss: 0.4944 - accuracy: 0.7604
Epoch 249/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4918 - accuracy: 0.7760
Epoch 250/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4712 - accuracy: 0.7799
Epoch 251/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4734 - accuracy: 0.7682
Epoch 252/300
26/26 [==============================] - 0s 1ms/step - loss: 0.4803 - accuracy: 0.7708
Ep

In [17]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 843us/step - loss: 0.4586 - accuracy: 0.7773
Accuracy: 77.73


In [18]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions 
rounded = [round(x[0]) for x in predictions]


In [19]:
import numpy as np


classes_x=np.argmax(predictions,axis=1)


for i in range(10):
    print('%s => %d (expected %d)' % (X[i].tolist(), classes_x[i], y[i]))


[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] => 0 (expected 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] => 0 (expected 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] => 0 (expected 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] => 0 (expected 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] => 0 (expected 1)


## Challenge 2:
How can you tune the model to improve the accuracy to be higher than what you have obtained?

In [20]:
# fit the keras model on the dataset
model.fit(X, y, epochs=1500, batch_size=60)

Epoch 1/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4572 - accuracy: 0.7760
Epoch 2/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4617 - accuracy: 0.7839
Epoch 3/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4551 - accuracy: 0.7786
Epoch 4/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4570 - accuracy: 0.7682
Epoch 5/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4624 - accuracy: 0.7773
Epoch 6/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4521 - accuracy: 0.7865
Epoch 7/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4560 - accuracy: 0.7773
Epoch 8/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4534 - accuracy: 0.7852
Epoch 9/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4519 - accuracy: 0.7852
Epoch 10/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4555 - accuracy: 0.7878

13/13 [==============================] - 0s 1ms/step - loss: 0.4582 - accuracy: 0.7747
Epoch 83/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4578 - accuracy: 0.7839
Epoch 84/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4535 - accuracy: 0.7904
Epoch 85/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4661 - accuracy: 0.7826
Epoch 86/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4537 - accuracy: 0.7695
Epoch 87/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4590 - accuracy: 0.7812
Epoch 88/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4509 - accuracy: 0.7799
Epoch 89/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4469 - accuracy: 0.7852
Epoch 90/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4451 - accuracy: 0.7852
Epoch 91/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4467 - accuracy: 0.7852
Epoc

13/13 [==============================] - 0s 2ms/step - loss: 0.4407 - accuracy: 0.7786
Epoch 163/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4472 - accuracy: 0.7812
Epoch 164/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4406 - accuracy: 0.7930
Epoch 165/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4386 - accuracy: 0.7956
Epoch 166/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4428 - accuracy: 0.7878
Epoch 167/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4440 - accuracy: 0.7839
Epoch 168/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4382 - accuracy: 0.7839
Epoch 169/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4417 - accuracy: 0.7852
Epoch 170/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4439 - accuracy: 0.7930
Epoch 171/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4469 - accuracy: 0.

13/13 [==============================] - 0s 1ms/step - loss: 0.4396 - accuracy: 0.7904
Epoch 243/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4367 - accuracy: 0.7904
Epoch 244/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4401 - accuracy: 0.7878
Epoch 245/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4361 - accuracy: 0.7943
Epoch 246/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4342 - accuracy: 0.7969
Epoch 247/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4457 - accuracy: 0.7839
Epoch 248/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4350 - accuracy: 0.7930
Epoch 249/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4318 - accuracy: 0.7956
Epoch 250/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4347 - accuracy: 0.7812
Epoch 251/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4377 - accuracy: 0.

13/13 [==============================] - 0s 1ms/step - loss: 0.4333 - accuracy: 0.7943
Epoch 323/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4430 - accuracy: 0.7878
Epoch 324/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4405 - accuracy: 0.7865
Epoch 325/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4390 - accuracy: 0.7969
Epoch 326/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4356 - accuracy: 0.7956
Epoch 327/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4418 - accuracy: 0.7878
Epoch 328/1500
13/13 [==============================] - 0s 916us/step - loss: 0.4349 - accuracy: 0.7956
Epoch 329/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4334 - accuracy: 0.7786
Epoch 330/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.7969
Epoch 331/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4322 - accurac

...

13/13 [==============================] - 0s 1ms/step - loss: 0.4060 - accuracy: 0.7969
Epoch 1121/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4100 - accuracy: 0.7982
Epoch 1122/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4147 - accuracy: 0.8047
Epoch 1123/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4153 - accuracy: 0.8021
Epoch 1124/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4047 - accuracy: 0.7982
Epoch 1125/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4039 - accuracy: 0.8151
Epoch 1126/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3951 - accuracy: 0.8086
Epoch 1127/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4004 - accuracy: 0.8060
Epoch 1128/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4064 - accuracy: 0.8008
Epoch 1129/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4

13/13 [==============================] - 0s 1ms/step - loss: 0.4125 - accuracy: 0.8138
Epoch 1200/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4035 - accuracy: 0.7995
Epoch 1201/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4012 - accuracy: 0.8060
Epoch 1202/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4036 - accuracy: 0.8073
Epoch 1203/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4072 - accuracy: 0.8034
Epoch 1204/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4087 - accuracy: 0.7956
Epoch 1205/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4063 - accuracy: 0.8034
Epoch 1206/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4010 - accuracy: 0.7995
Epoch 1207/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4012 - accuracy: 0.8086
Epoch 1208/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4040 - acc

13/13 [==============================] - 0s 1ms/step - loss: 0.4021 - accuracy: 0.8060
Epoch 1279/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3974 - accuracy: 0.8060
Epoch 1280/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3956 - accuracy: 0.8060
Epoch 1281/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4030 - accuracy: 0.8008
Epoch 1282/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.3979 - accuracy: 0.7982
Epoch 1283/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3998 - accuracy: 0.7995
Epoch 1284/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4032 - accuracy: 0.8021
Epoch 1285/1500
13/13 [==============================] - 0s 833us/step - loss: 0.4066 - accuracy: 0.8034
Epoch 1286/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.4025 - accuracy: 0.7904
Epoch 1287/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4127 

13/13 [==============================] - 0s 1ms/step - loss: 0.4092 - accuracy: 0.7956
Epoch 1358/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4032 - accuracy: 0.7995
Epoch 1359/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4167 - accuracy: 0.7969
Epoch 1360/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4071 - accuracy: 0.8177
Epoch 1361/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4070 - accuracy: 0.8034
Epoch 1362/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4002 - accuracy: 0.7930
Epoch 1363/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4016 - accuracy: 0.8021
Epoch 1364/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4013 - accuracy: 0.7969
Epoch 1365/1500
13/13 [==============================] - 0s 917us/step - loss: 0.4008 - accuracy: 0.8060
Epoch 1366/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3977 - a

13/13 [==============================] - 0s 1ms/step - loss: 0.3970 - accuracy: 0.8034
Epoch 1437/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.3989 - accuracy: 0.8060
Epoch 1438/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4006 - accuracy: 0.8060
Epoch 1439/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.3952 - accuracy: 0.8021
Epoch 1440/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3967 - accuracy: 0.8047
Epoch 1441/1500
13/13 [==============================] - 0s 2ms/step - loss: 0.4026 - accuracy: 0.7995
Epoch 1442/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3996 - accuracy: 0.8047
Epoch 1443/1500
13/13 [==============================] - 0s 1000us/step - loss: 0.3971 - accuracy: 0.8099
Epoch 1444/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.4061 - accuracy: 0.8034
Epoch 1445/1500
13/13 [==============================] - 0s 1ms/step - loss: 0.3990 - 

In [21]:
accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy[1]*100))

8/8 [==============================] - 0s 1ms/step - loss: 0.3950 - accuracy: 0.7992
Accuracy: 79.92


## Challenge 3:
How can you save the model and use it again for prediction directly without (re-)training?

In [22]:
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

# load model
model = load_model('model.h5')

# summarize model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12)                108       
                                                                 
 dense_4 (Dense)             (None, 8)                 104       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [24]:
accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy[1]*100))

8/8 [==============================] - 0s 857us/step - loss: 0.3950 - accuracy: 0.7992
Accuracy: 79.92
